#Cloud Constable
---
##Web Scraping Bad Guy Chats/SMS

### Step 1 - Scrape the chat links

Import needed libraries

In [0]:
import requests
import bs4
import pandas as pd
import numpy as np
import string

Grab the main archive page and create a BS object

In [0]:
root_url = "http://www.perverted-justice.com"
response = requests.get(root_url + "/?con=full")
soup = bs4.BeautifulSoup(response.text, "html.parser")

Create a BS object with all of our tags

In [0]:
tags = soup.find(id="mainbox").find_all(["a","b"])

Get lists for the urls, names, and states and add them to a dataframe. **Note**: parsing the tags breaks down at line 464, so we do it in two parts

In [0]:
# get lists of urls, names, and states
urls=[root_url + tag.get('href')[1:] for tag in tags[2:468:6]+tags[469::6]]
names=[tag.get_text() for tag in tags[2:468:6]+tags[469::6]]
states=[tag.get_text().split(',')[1] for tag in tags[3:468:6]+tags[470::6]]

# create a dataframe of the lists
linksDF = pd.DataFrame(
    {'Name': names,
     'State': states,
     'URL': urls
    })

print (linksDF.shape[0], 'cases with', linksDF.shape[1], 'attrbutes')

623 cases with 3 attrbutes


In [0]:
linksDF.head(5)

Connect to google sheets

In [0]:
!pip install --upgrade -q gspread

from google.colab import auth
auth.authenticate_user()

import gspread
from oauth2client.client import GoogleCredentials

gc = gspread.authorize(GoogleCredentials.get_application_default())

Write the pandas dataframe to a google sheet

In [0]:
!pip install --upgrade -q gspread_dataframe
import gspread_dataframe as gd

# create the sheet (uncomment if not there already)
#sh = gc.create('Bad Guy URLs')

In [0]:
# open google sheet and write
ws = gc.open("Bad Guy URLs").worksheet("Sheet1")
gd.set_with_dataframe(ws, linksDF)

NameError: ignored

### Step 2 - Scrape the chat logs

Load the chat log URLs from google sheets

In [0]:
# open the google sheet and read it in to a DF
!pip install --upgrade -q gspread_dataframe
import gspread_dataframe as gd
ws = gc.open("Bad Guy URLs").worksheet("Sheet1")
linksDF = gd.get_as_dataframe(ws)
linksDF.head(5)

,Name,State,URL
0,Josh Mcclendon,Alabama,http://www.perverted-justice.com/?archive=Josh...
1,mg0942,California,http://www.perverted-justice.com/?archive=mg0942
2,jackjohnsons7,Alabama,http://www.perverted-justice.com/?archive=jack...
3,louisianamaster1983,Louisiana,http://www.perverted-justice.com/?archive=loui...
4,Nice Guy,Florida,http://www.perverted-justice.com/?archive=Nice...


Create an empty DF to store the chat logs and loop through the first 3 chats-- something's wrong on the subsequent chats

In [0]:
# Create an empty DF for all chat logs 
allChat = pd.DataFrame(columns=["Name","Sliminess","State","Text"])

# Loop through links and scrape
for i in range(linksDF.shape[0]):

  # Get the name and state
  name  = linksDF.iloc[i]["Name"]
  state = linksDF.iloc[i]["State"]

  print ("Scraping",name,"from",state,"chat")
  response = requests.get(linksDF.iloc[i]["URL"])
  soup = bs4.BeautifulSoup(response.text, "html.parser")

  # Get the sliminess score
  try:
    sliminess = soup.find("big", {"id": "reallyBig"}).get_text().split()[2]
  except:
    pass
  
  # get all bold tags
  boldTags = np.array([b.get_text() for b in soup.find_all("b")])

  # get freqs
  unique, counts = np.unique(boldTags, return_counts=True)
  freqs = np.asarray((unique, counts)).T

  # get bad guy tag labels where there are more than five occurences
  nameLabels = freqs[freqs[:,1].astype(int)>5][:,0]
  print ('Bad guy labels:', nameLabels)
  
  # Create NP array for the chat
  chatLog = np.array([])
  
  # Find two types of chats
  try:
    chatRows = soup.find_all("b",string=nameLabels)
    IMRows = soup.find_all("span",{'class' : 'blueBold'})
  except:
    pass
  
  # Get all bad guy tags and return the text following it in the dialog
  for b in chatRows:
    try:
      chatLog = np.append ( chatLog, str(b.next_sibling.string) )
    except:
      pass
    
  # Get all bad guy IM lines
  for im in IMRows:
    try:
      chatLog = np.append ( chatLog, str(im.next_sibling.string) )
    except:
      pass
  
  # Get the number of lines
  chatLines = chatLog.shape[0]
  print (chatLines,"lines of chat")

  # only write to DF if there are lines of chat
  if chatLines > 0:
    # create a dataframe of the chat
    chatDF = pd.DataFrame(
      {'Name': np.repeat (name,chatLines),
       'State': np.repeat (state,chatLines),
       'Sliminess': np.repeat (sliminess,chatLines),
       'Text': chatLog
      })
  
    # Append the chat log to the full DF
    allChat = allChat.append(chatDF)

Scraping Josh Mcclendon from  Alabama chat
Bad guy labels: ['Josh Mcclendon' 'Josh Mcclendon Phone']
363 lines of chat
Scraping mg0942 from  California chat
Bad guy labels: ['Michael Garcia']
149 lines of chat
Scraping jackjohnsons7 from  Alabama chat
Bad guy labels: ['Bob Dylan' 'Bob Dylan Phone']
118 lines of chat
Scraping louisianamaster1983 from  Louisiana chat
Bad guy labels: []
427 lines of chat
Scraping Nice Guy from  Florida chat
Bad guy labels: ['Nice Guy' 'Nice Guy Phone']
288 lines of chat
Scraping luv2licku68 from  Utah chat
Bad guy labels: []
1854 lines of chat
Scraping Kent Grant from  California chat
Bad guy labels: ['Kent Grant' 'Kent Grant Phone']
76 lines of chat
Scraping J H from  Alabama chat
Bad guy labels: ['J H' 'J H Phone']
211 lines of chat
Scraping paulcarder86 from  Indiana chat
Bad guy labels: ['paulcarder86 Phone' 'renegade62']
668 lines of chat
Scraping innocentz6197@sbcglobal.net from  Connecticut chat
Bad guy labels: ['Gabriel Dos Santos:' 'innocentz6197

Bad guy labels: []
3142 lines of chat
Scraping bmichigan69 from  Michigan chat
Bad guy labels: []
231 lines of chat
Scraping krazythug_692000 from  Michigan chat
Bad guy labels: []
1105 lines of chat
Scraping recon101_2000 from  Michigan chat
Bad guy labels: []
141 lines of chat
Scraping  wl72971 from  Michigan chat
Bad guy labels: []
708 lines of chat
Scraping georgia_emti from  Georgia chat
Bad guy labels: []
1101 lines of chat
Scraping jjj_200925 from  Wisconsin chat
Bad guy labels: []
240 lines of chat
Scraping hunglowilove69 from  California chat
Bad guy labels: []
257 lines of chat
Scraping darkprince666_2006 from  Louisiana chat
Bad guy labels: []
129 lines of chat
Scraping raidersdawg5 from  Wisconsin chat
Bad guy labels: []
926 lines of chat
Scraping lovesource7 from  Florida chat
Bad guy labels: ['lovesource7']
7278 lines of chat
Scraping jaguarsd69 from  New York chat
Bad guy labels: []
170 lines of chat
Scraping philknowlan from  Michigan chat
Bad guy labels: []
2358 lines 

Bad guy labels: []
1727 lines of chat
Scraping malhotra.mohit from  Georgia chat
Bad guy labels: ['malhotra.mohit']
4934 lines of chat
Scraping tito_92503 from  California chat
Bad guy labels: []
1347 lines of chat
Scraping dukered2000 from  Michigan chat
Bad guy labels: []
3609 lines of chat
Scraping dipoyoy from  OH chat
Bad guy labels: []
2927 lines of chat
Scraping hel_bomber from  Michigan chat
Bad guy labels: []
1798 lines of chat
Scraping jeremykeister from  New Jersey chat
Bad guy labels: []
344 lines of chat
Scraping charlie_e44 from  Alabama chat
Bad guy labels: []
5190 lines of chat
Scraping hungindian_09 from  West Virginia chat
Bad guy labels: []
884 lines of chat
Scraping bmars1411 and skydivescout82 from  Utah chat
Bad guy labels: ['bmars1411' 'skydivescout82']
266 lines of chat
Scraping jackman_9682 from  Oklahoma chat
Bad guy labels: ['jackman_9682']
863 lines of chat
Scraping geo_allino from  Michigan chat
Bad guy labels: ['geo_allino']
6183 lines of chat
Scraping tru

4355 lines of chat
Scraping mkrauss0460 from  Michigan chat
Bad guy labels: []
442 lines of chat
Scraping cantonguy4fun / cantonguylookin from  Georgia chat
Bad guy labels: ['cantonguy4fun' 'cantonguylookin']
2150 lines of chat
Scraping aggresk8er from  Utah chat
Bad guy labels: []
4931 lines of chat
Scraping justmetoddray from  Michigan chat
Bad guy labels: ['justmetoddray']
751 lines of chat
Scraping davekruz2003 from  Florida chat
Bad guy labels: ['davekruz2003']
581 lines of chat
Scraping orl00fun from  Florida chat
Bad guy labels: ['orl00fun']
605 lines of chat
Scraping atlanta.italian from  Georgia chat
Bad guy labels: ['atlanta.italian' 'italian_in_atlanta ' 'tealitalianeyes']
1742 lines of chat
Scraping theoldmonk_4_all from  Pennsylvania chat
Bad guy labels: []
2353 lines of chat
Scraping Salsakewl from  California chat
Bad guy labels: []
4 lines of chat
Scraping nhbfullcontact21 from  California chat
Bad guy labels: []
171 lines of chat
Scraping n_karov from  Vermont chat
Bad

Bad guy labels: []
184 lines of chat
Scraping victor_wells2002 from  California chat
Bad guy labels: []
683 lines of chat
Scraping Hibuddywazsup from  New York chat
Bad guy labels: ['Hi buddy waz sup']
317 lines of chat
Scraping zigdog2k3 from  New York chat
Bad guy labels: []
186 lines of chat
Scraping nj_dom_4_f from  New Jersey chat
Bad guy labels: []
3299 lines of chat
Scraping ussoldier2281 / lowcamaro97 from  Illinois chat
Bad guy labels: ['lowcamaro97' 'ussoldier2281']
309 lines of chat
Scraping FirstAndGrand  from  Florida chat
Bad guy labels: []
388 lines of chat
Scraping ikeman_04  from  New Jersey chat
Bad guy labels: []
688 lines of chat
Scraping figwithbqe from  New York chat
Bad guy labels: ['figwithbqe']
3106 lines of chat
Scraping loverboy212x from  New Jersey chat
Bad guy labels: []
1485 lines of chat
Scraping njdavenj1 from  New Jersey chat
Bad guy labels: []
904 lines of chat
Scraping lovermangenuis from  New Jersey chat
Bad guy labels: []
639 lines of chat
Scraping 

...

Bad guy labels: []
212 lines of chat
Scraping Printo316 from  Florida chat
Bad guy labels: []
467 lines of chat
Scraping dble_d1 from  Wisconsin chat
Bad guy labels: []
469 lines of chat
Scraping brian_e024 from  Florida chat
Bad guy labels: []
325 lines of chat
Scraping pitbulldavid2001 / caliguy4u_2005 from  California chat
Bad guy labels: ['caliguy4u_2005']
3587 lines of chat
Scraping iurutherford from  Ohio chat
Bad guy labels: []
1602 lines of chat
Scraping Adamou217 from  Texas chat
Bad guy labels: ['adamou217' '{adams217}']
211 lines of chat
Scraping lee_greer74 from  FL chat
Bad guy labels: ['lee_greer74']
867 lines of chat
Scraping mr6317 from  California chat
Bad guy labels: []
1601 lines of chat
Scraping hey_malen from  Illinois chat
Bad guy labels: []
209 lines of chat
Scraping jackenit_6in from  Nevada chat
Bad guy labels: []
4229 lines of chat
Scraping desiredalex from  California chat
Bad guy labels: []
470 lines of chat
Scraping bravesfan34581 from  California chat
Bad 

Bad guy labels: []
282 lines of chat
Scraping ricks_talented_tongue from  California chat
Bad guy labels: []
172 lines of chat
Scraping welfare_isforwhitefolks from  North Carolina chat
Bad guy labels: []
322 lines of chat
Scraping magic4isl2002 from  San Diego chat
Bad guy labels: []
869 lines of chat
Scraping latino_626 from  California chat
Bad guy labels: ['latino_626']
405 lines of chat
Scraping farzadboy2003 from  California chat
Bad guy labels: []
390 lines of chat
Scraping WebDev2 from  Calif. chat
Bad guy labels: ['{WebDev2} ']
69 lines of chat
Scraping chinaski818 from  California chat
Bad guy labels: []
129 lines of chat
Scraping godofwineof3eb from  California chat
Bad guy labels: []
272 lines of chat
Scraping your_lil_nene from  California chat
Bad guy labels: []
573 lines of chat
Scraping mxracer_126  from  California chat
Bad guy labels: []
328 lines of chat
Scraping clitlicker42303 from  Kentucky chat
Bad guy labels: []
437 lines of chat
Scraping jim90020 from  Californ

Bad guy labels: []
384 lines of chat
Scraping abraxisreborn from  Wisconsin chat
Bad guy labels: []
452 lines of chat
Scraping parknride_469 from  Ohio chat
Bad guy labels: ['parknride_469']
185 lines of chat
Scraping goodbody1330 from  OH chat
Bad guy labels: []
163 lines of chat
Scraping lowvoltage18 from  California chat
Bad guy labels: ['lowvoltage18']
186 lines of chat
Scraping stoned_party from  California chat
Bad guy labels: ['stoned_party']
3200 lines of chat
Scraping Crazy4theNATS from  District of Columbia chat
Bad guy labels: []
161 lines of chat
Scraping thomascoffen from  Florida chat
Bad guy labels: []
459 lines of chat
Scraping jmv91373 from  CA chat
Bad guy labels: ['jmv91373']
816 lines of chat
Scraping singletxguyforfun from  Ohio chat
Bad guy labels: ['singletxguyforfun']
107 lines of chat
Scraping hotplushung from  Kentucky chat
Bad guy labels: []
442 lines of chat
Scraping fargo1982 from  North Dakota chat
Bad guy labels: []
236 lines of chat
Scraping i_luv_the_pu

Scraping wesleybrannen82 from  California chat
Bad guy labels: []
291 lines of chat
Scraping manofdarkneedsl951 from  Ohio chat
Bad guy labels: []
94 lines of chat
Scraping alonzo403 from  Ohio chat
Bad guy labels: []
132 lines of chat
Scraping texassailor04 from  Virginia chat
Bad guy labels: []
156 lines of chat
Scraping hum_366 from  Texas chat
Bad guy labels: ['[Link]']
791 lines of chat
Scraping likuososweet from  Nevada chat
Bad guy labels: ['[Link]' 'waynev_1999']
751 lines of chat
Scraping navyboy_usn from  Washington chat
Bad guy labels: []
344 lines of chat
Scraping JDC074 and Sgps malaysia from  California chat
Bad guy labels: []
4 lines of chat
Scraping goodguyportsmouthva from  Virginia chat
Bad guy labels: ['goodguyportsmouthva']
413 lines of chat
Scraping Justinnbos from  Massachusetts chat
Bad guy labels: []
119 lines of chat
Scraping KCROWOO from  Virginia chat
Bad guy labels: []
360 lines of chat
Scraping aladiesmasseur from  California chat
Bad guy labels: []
174 lin

Bad guy labels: []
221 lines of chat
Scraping b080387 from  Minnesota chat
Bad guy labels: []
162 lines of chat
Scraping Arthinice from  Arkansas chat
Bad guy labels: []
72 lines of chat
Scraping fleet_captain_jaime_wolfe from  Illinois chat
Bad guy labels: []
1346 lines of chat
Scraping atonomous2000 from  Michigan chat
Bad guy labels: []
157 lines of chat
Scraping the_donjuan02 from  Colorado chat
Bad guy labels: []
313 lines of chat


Now that we're done, let's do a sanity check on the text and get some stats

In [0]:
allChat.head(5)

,Name,Sliminess,State,Text
0,Josh Mcclendon,3.64,Alabama,- Hey beautiful I see That we obviously don't...
1,Josh Mcclendon,3.64,Alabama,- 23
2,Josh Mcclendon,3.64,Alabama,- It's on my profile btw lol
3,Josh Mcclendon,3.64,Alabama,- No I been single since like jan
4,Josh Mcclendon,3.64,Alabama,- But 14 danger wasn't expecting that one lol


In [0]:
allChat.tail(5)

,Name,Sliminess,State,Text
308,the_donjuan02,4.47,Colorado,(1:34:33 AM): hmmmm
309,the_donjuan02,4.47,Colorado,(1:34:39 AM): want to on the phone?
310,the_donjuan02,4.47,Colorado,(1:35:11 AM): ok
311,the_donjuan02,4.47,Colorado,(1:36:05 AM): ok
312,the_donjuan02,4.47,Colorado,(2:11:28 AM): u there??


In [0]:
print ( allChat.shape[0], "total lines of SMS/chat/IM", "with", allChat.shape[1], "attributes." )

679539 total lines of SMS/chat/IM with 4 attributes.


Finally, we download the final chat log, doesn't fit in a google sheet :-(

In [0]:
from google.colab import files
allChat.to_csv("BadGuyChat.tab", sep="\t", index=False)
files.download('BadGuyChat.tab')